In [2]:
import pandas as pd
import re

# Exploratory analysis

The objective of this document was to poke the dataset in ways to gather insights to build a recommendation engine.

Here i present a summary of the insights, and finish presenting the an initial strategy to be utilized on the data:

The __movie__ database has genre which is an neutral input to segment the movies.

there is also the date the movie was relased, although it's sometimes missing. in order to get this data if needed, I might use the __links__ database to enrich the data with imdb's dataset, that has the release date.

the tags dataset is important to relate the users beyond their rating. but the tags are not objective like the genre. in order to clean the data, I will diminish the number of titles using __bag of words__, followed by a clustering algorithm.

# movies dataset

the preprocessing step will focus in expanding the genres column in single genre rows.
Would also be nice to extract the info of the year the title was launched.

In [4]:
movies = pd.read_csv("ml-20m/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings = pd.read_csv("ml-20m/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


## Controversy

In order to have a broader view of the dataset, I selected an example with a high variety of ratings, and that was among the most rated movies. MovieId 344: "Ace Ventura: Pet Detective (1994)". Perfect.

In [11]:
most_viewed_ratings = ratings.groupby("movieId")

In [16]:
most_viewed_ratings.agg({'rating':['mean', 'std', 'count']}).sort_values([('rating', 'count')], ascending=False).head(50)

rating                 
             mean       std  count
movieId                           
296      4.174231  0.976076  67310
356      4.029000  0.948018  66172
318      4.446990  0.717525  63366
593      4.177057  0.842421  63299
480      3.664741  0.927818  59715
260      4.190672  0.918522  54502
110      4.042534  0.955058  53769
589      3.931954  0.903087  52244
2571     4.187186  0.887929  51334
527      4.310175  0.825421  50054
1        3.921240  0.889012  49695
457      3.985690  0.772525  49581
150      3.868598  0.850971  47777
780      3.370962  1.059615  47048
50       4.334372  0.756783  47006
1210     4.004622  0.909598  46839
592      3.402365  0.833750  46054
1196     4.188202  0.879710  45313
2858     4.155934  0.897111  44987
32       3.898055  0.867135  44980
590      3.728465  0.960581  44208
1198     4.219009  0.784264  43295
608      4.112359  0.920090  43272
47       4.053493  0.870280  43249
380      3.491149  0.931578  43159
588      3.680871  0.912677  41842
377      3.493203  0.936198  41562
1270     3.924142  0.838150  41426
858      4.364732  0.839877  41355
2959     4.227123  0.870948  40106
2762     4.054999  0.828826  39028
364      3.779090  0.930331  38991
344      2.982342  1.106229  38226
4993     4.137925  0.948095  37553
648      3.388599  0.889920  37127
2028     4.064417  0.895753  37110
1580     3.558319  0.902495  35580
595      3.673302  0.979678  35138
500      3.386267  0.950245  34392
367      3.174718  0.979632  34384
5952     4.107521  0.931838  33947
165      3.489025  0.887390  33940
597      3.420428  0.974559  33900
1240     3.896392  0.874660  33429
1136     4.174146  0.873922  33024
3578     3.952248  0.904940  32878
153      2.894727  0.949923  32867
1097     3.754918  0.953475  32685
1197     4.176732  0.870416  32586
736      3.196249  1.003010  32446

In [13]:
movies[movies['movieId']==344]

,movieId,title,genres
340,344,Ace Ventura: Pet Detective (1994),Comedy


# Tags Dataset 

The tags dataset is an interesting one. 

With it, it is clear to distinguish some of the reasons the users have to assign each title's rating. 

Rating in this case can be an emotional choice, wether the user has a deep connecion with the title, such as its his childhood favorite movie or in the opposite case, has elements such as horror that can disengage some audiences.

there are some problems though, such as the similarity of tags, such as "good", "Good" and "very good". i intend to treat that with a bag of words approach.

In [6]:
tags = pd.read_csv("ml-20m/tags.csv")
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [7]:
len(tags.tag.unique())

38644

In [8]:
tag_ratings = pd.merge(tags, ratings[['userId', 'movieId', 'rating']], on=['userId', 'movieId'])

## How do you feel about it?

the "very dumb" tag is present in both the lowest rating and the highest rating. this demonstrates the disparity of people's tastes. but what feeling each tag convey on average?

In [9]:
tag_ratings[tag_ratings['movieId'] == 344].sort_values('rating').head()

,userId,movieId,tag,timestamp,rating
388687,137805,344,Gross-out,1241235261,0.5
95261,26855,344,Tone Loc,1371095117,0.5
95260,26855,344,Jim Carrey,1367815816,0.5
19172,5036,344,Jim Carrey,1259972321,0.5
11784,3596,344,very dumb,1290308441,0.5


In [10]:
tag_ratings[tag_ratings['movieId'] == 344].sort_values('rating', ascending=False).head()

,userId,movieId,tag,timestamp,rating
285642,103125,344,silly fun,1303414195,5.0
285634,103125,344,Gross-out,1303414175,5.0
285645,103125,344,very dumb,1303414200,5.0
285644,103125,344,Tom Shadyac,1303414198,5.0
285643,103125,344,Stupid as Hell,1303414198,5.0


## was it really that funny?
it is also interesting to note that "not funny" and "very funny" are half a star away.
what could this mean? 

an initial guess is that even though people would not recommend the movie (if it would be classified in a binary "like" or "didn't like" methodology) they stil tend to give at least a three star rating.

In [65]:
tag_ratings[tag_ratings['movieId'] == 344].groupby('tag').median()['rating'].sort_values()

tag
jim carrey                     1.00
gross-out                      1.00
Want                           1.50
Gross-out                      2.00
Stupid as Hell                 2.00
Tone Loc                       2.50
humor for the narrow-minded    2.50
very dumb                      2.50
bad ending                     3.00
goofy                          3.00
Dumb                           3.00
Jim Carrey being Jim Carrey    3.50
own                            3.50
Jim Carrey                     3.50
not funny                      3.50
crude                          3.50
sex jokes                      3.50
silly fun                      3.50
quick change artist            4.00
private detective              4.00
crude humor                    4.00
very funny                     4.00
comedy                         4.00
Tom Shadyac                    4.00
rate up                        4.25
Courteney Cox                  4.25
hilarious                      4.50
detective               